In [168]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import json
from datetime import datetime
import psycopg2
from cryptography.fernet import Fernet

In [169]:
## Encryption
message = "my name is Shyam"
key = Fernet.generate_key()
fernet = Fernet(key)

In [141]:
# Create a sqs client
sqs = boto3.client('sqs', region_name='us-west-2',endpoint_url='http://localhost:4566',config=Config(signature_version=UNSIGNED))
queue_url = 'http://localhost:4566/000000000000/login-queue'
# sqs receive message
response = sqs.receive_message(
    QueueUrl=queue_url,
    MaxNumberOfMessages=2,
    WaitTimeSeconds=5
)


In [151]:
sqs_message = response["Messages"][0]

In [152]:
sqs_message

{'MessageId': 'aa1747fb-768d-4196-bfdc-e4901c92a0ec',
 'ReceiptHandle': 'MDU1ZTA1ZmQtNzhmMi00ZWNmLWI1NjQtMmI3ODc2YTEzMWUzIGFybjphd3M6c3FzOnVzLWVhc3QtMTowMDAwMDAwMDAwMDA6bG9naW4tcXVldWUgYWExNzQ3ZmItNzY4ZC00MTk2LWJmZGMtZTQ5MDFjOTJhMGVjIDE2ODkyMjgxMDcuMjU2NjY0NQ==',
 'MD5OfBody': 'e4f1de8c099c0acd7cb05ba9e790ac02',
 'Body': '{"user_id": "424cdd21-063a-43a7-b91b-7ca1a833afae", "app_version": "2.3.0", "device_type": "android", "ip": "199.172.111.135", "locale": "RU", "device_id": "593-47-5928"}'}

### Message Data structure

In [161]:
class Message:
    
    def __init__(self,user_id,app_version, device_type, ip, locale, device_id):
        self.user_id = user_id
        self.app_version = "".join(app_version.split("."))
        self.device_type = device_type
        self.ip = fernet.encrypt(ip.encode()).decode()
        self.locale = locale
        self.device_id  = fernet.encrypt(device_id.encode()).decode()   
        self.created_date = datetime.today().strftime('%Y-%m-%d')
        

In [162]:
m = Message(**json.loads(sqs_message["Body"]))

### Dat

In [163]:
params = {
    "host":"localhost",
    "database":"postgres",
    "user":"postgres",
    "password":"postgres",
    "port":5432
}

In [164]:
#!/usr/bin/python
def connectAndRun(command):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        print('Connecting to the PostgreSQL database...')
        
        conn = psycopg2.connect(**params)
        cur = conn.cursor()
        
        cur.execute(command)
        conn.commit()    
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


In [165]:
command = f"insert into user_logins(user_id, device_type, masked_ip, masked_device_id,locale , app_version, create_date) values ('{m.user_id}','{m.device_type}','{m.ip}','{m.device_id }','{m.locale}','{m.app_version}', '{m.created_date}');"


In [166]:
command

"insert into user_logins(user_id, device_type, masked_ip, masked_device_id,locale , app_version, create_date) values ('424cdd21-063a-43a7-b91b-7ca1a833afae','android','gAAAAABkr5O9VdclC97ERUlVrSTAMJZmr13YdDsuxavGwwCtr6dDv9Mp5Yx15Dpsa3DaD2HmFoS35GvjP_tC_OkI3LgZIbbDXg==','gAAAAABkr5O9jSF6GfHnjZZgSd5UjRMvCFqMJP7x5w436RafTURGeysQpnO4TRISHXGWopSJezu-83WyBnJPoB2-FyTJYZQjcg==','RU','230', '2023-07-13');"

In [167]:
connectAndRun(command)

Connecting to the PostgreSQL database...
Database connection closed.
